<a href="https://colab.research.google.com/github/joostnjns/Google-Earth-Engine/blob/master/DownloadLandsatSebha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Script using the Google Earth Engine API to download Landsat imagery.**

In [0]:
# Run only at first run: install python API
!pip install earthengine-api

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/sgECD9CBvTwfNXbmYahxnXQGUxVqthMEEXzURl7Mql3Jb2exuwwGB3M

Successfully saved authorization token.


In [1]:
import ee
ee.Authenticate()

# Get other packages
import matplotlib.pyplot as plt
from IPython.display import Image
import pprint
import folium
import time

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/swGhyI10Zg61ZVmwSaoWf6B7HrTECh26JtmebPdsA6d8a0moYACqsTo

Successfully saved authorization token.


In [0]:
# Download a Landsat image based on some parameters

# LANDSAT/LT05/C01/T1_SR meaning:
# Look into the ImageCollections of Landsat. 
# In our case use Landsat 5 sensors. 
# USGS processes them in 'collections', with collection 1 (C01) as the only option.
# To get the surface reflectane datasat for collection 1, use T1_SR (tier 1)

In [0]:
# Set AOI for imagery
aoi = ee.Geometry.Rectangle([14.24, 26.92, 14.58, 27.14])

# Create scene from image with least cloud cover in collection
collection = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').filterDate('1995-01-10', '1995-01-30').filterBounds(aoi).filterMetadata('IMAGE_QUALITY', 'equals', 9).filterMetadata('CLOUD_COVER_LAND', 'equals', 0)  # Collection containing potentially multiple images
RGBscene = collection.sort('CLOUD COVER').first().select(['B4', 'B3', 'B2'])  # RGB image
fullscene = collection.sort('CLOUD COVER').first().select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'])  # all bands
atm_opac = collection.sort('CLOUD COVER').first().select(['sr_atmos_opacity'])  # atmospheric opacity
sr_cloud_qa = collection.sort('CLOUD COVER').first().select(['sr_cloud_qa'])  # surface reflectance cloud quality: indicating which pixels might be affected by instrument artifacts or subject to cloud contamination

#RGBsceneUrl = RGBscene.getThumbUrl()
#sceneUrl = fullscene.getThumbUrl({'min': 0.0, 'max': 0.3})

In [184]:
test = fullscene.getThumbUrl(fullscene)


TypeError: ignored

In [178]:
"""
# If you just want to plot the image
#sceneUrl = scene.getThumbUrl()
#Image(url=sceneUrl)
"""
map = folium.Map(location=[27.05, 14.45], zoom_start=10, height=500)

folium.raster_layers.ImageOverlay(
    image=RGBscene,
    bounds=[[26.92,14.24],[27.14, 14.58]],
    colormap=lambda x: (1, 0, x, x),#R,G,B,alpha
).add_to(map)
"""map.add_child(folium.LayerControl())"""
map

TypeError: ignored

In [153]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
visParams = {'min':0, 'max':3000}

# Create a folium map object.
myMap = folium.Map(location=[27.05, 14.45], zoom_start=10, height=500)

# Add the elevation model to the map object.
myMap.add_ee_layer(sceneUrl, visParams, 'Sebha')

# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
display(myMap)

EEException: ignored

In [0]:
mapid = scene.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})

In [114]:
m = folium.Map(location=[27.05, 14.45], tiles = "Stamen Terrain", width = '30%', height = '50%', zoom_start = 12)
m.add_child(folium.raster_layers.ImageOverlay(sceneUrl, opacity=.7, bounds=[[26.92, 14.24], [27.14, 14.58]]))

m





In [120]:
scene.getInfo()


{'bands': [{'crs': 'EPSG:32635',
   'crs_transform': [30.0, 0.0, 338685.0, 0.0, -30.0, 1701915.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7781, 6991],
   'id': 'B1'},
  {'crs': 'EPSG:32635',
   'crs_transform': [30.0, 0.0, 338685.0, 0.0, -30.0, 1701915.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7781, 6991],
   'id': 'B2'},
  {'crs': 'EPSG:32635',
   'crs_transform': [30.0, 0.0, 338685.0, 0.0, -30.0, 1701915.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7781, 6991],
   'id': 'B3'},
  {'crs': 'EPSG:32635',
   'crs_transform': [30.0, 0.0, 338685.0, 0.0, -30.0, 1701915.0],
   'data_type': {'max': 32767,
    'min': -32768,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7781, 6991],
   'id': 'B4'},
  {'crs': 'EPSG:32635',
   'crs_transform':

In [138]:
# Export the image, specifying scale and region.
##Export.image.toDrive({image: scene, description: 'imageToDriveExample', scale: 30, region: aoi});

task_config = {
    'scale': 30,
    'fileFormat': 'GeoTIFF',
    'skipEmptyTiles': True,
    'folder': 'testolder'
    }

task = ee.batch.Export.image.toDrive(scene, 'first test', **task_config)

task.start()

# Monitor the task.
while task.status()['state'] in ['READY', 'RUNNING']:
  print(task.status())
  time.sleep(10)
else:
  print(task.status())

{'id': 'BHUHMC7PCZUV567QJ5UKHJDG', 'state': 'READY', 'creation_timestamp_ms': 1572526052274, 'update_timestamp_ms': 1572526052274, 'description': 'first test', 'task_type': 'EXPORT_IMAGE'}


NameError: ignored

In [141]:
# Monitor the task.
while task.status()['state'] in ['READY', 'RUNNING']:
  print(task.status())
  time.sleep(10)
else:
  print(task.status())

{'id': 'BHUHMC7PCZUV567QJ5UKHJDG', 'state': 'RUNNING', 'creation_timestamp_ms': 1572526052274, 'update_timestamp_ms': 1572526062134, 'description': 'first test', 'task_type': 'EXPORT_IMAGE', 'start_timestamp_ms': 1572526061993, 'progress': 0.0}
{'id': 'BHUHMC7PCZUV567QJ5UKHJDG', 'state': 'RUNNING', 'creation_timestamp_ms': 1572526052274, 'update_timestamp_ms': 1572526062134, 'description': 'first test', 'task_type': 'EXPORT_IMAGE', 'start_timestamp_ms': 1572526061993, 'progress': 0.0}
{'id': 'BHUHMC7PCZUV567QJ5UKHJDG', 'state': 'RUNNING', 'creation_timestamp_ms': 1572526052274, 'update_timestamp_ms': 1572526062134, 'description': 'first test', 'task_type': 'EXPORT_IMAGE', 'start_timestamp_ms': 1572526061993, 'progress': 0.0}
{'id': 'BHUHMC7PCZUV567QJ5UKHJDG', 'state': 'RUNNING', 'creation_timestamp_ms': 1572526052274, 'update_timestamp_ms': 1572526062134, 'description': 'first test', 'task_type': 'EXPORT_IMAGE', 'start_timestamp_ms': 1572526061993, 'progress': 0.0}
{'id': 'BHUHMC7PCZUV